In [ ]:
!pip install openai
from openai import OpenAI
import os
import pandas as pd
from pandas import DataFrame
import requests
import time
from tqdm import tqdm
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

# OpenAI API 키값 설정하기
client = OpenAI(api_key="OPEN_AI_API_KEY")
df =  pd.read_csv('/content/drive/MyDrive/pknu/finance_data.csv', names=['sentence','kor_sentence'])


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df =  pd.read_csv('/content/drive/MyDrive/pknu/finance_data_test.csv', names=['sentence','kor_sentence'])
article_list = []

for article in df['sentence']:
  split= article.split()
  article_list.append(split)

df

,sentence,kor_sentence
labels,sentence,kor_sentence
neutral,"According to Gran, the company has no plans to...","Gran에 따르면, 그 회사는 회사가 성장하고 있는 곳이지만, 모든 생산을 러시아로..."
neutral,Technopolis plans to develop in stages an area...,테크노폴리스는 컴퓨터 기술과 통신 분야에서 일하는 회사들을 유치하기 위해 10만 평...
negative,The international electronic industry company ...,"국제 전자산업 회사인 엘코텍은 탈린 공장에서 수십 명의 직원을 해고했으며, 이전의 ..."
positive,With the new production plant the company woul...,새로운 생산공장으로 인해 회사는 예상되는 수요 증가를 충족시킬 수 있는 능력을 증가...
...,...,...
negative,RISING costs have forced packaging producer Hu...,가격 상승으로 인해 포장 생산업체인 허타마키는 햄프셔 공장에서 90명의 일자리를 잃...
negative,"`` However, the rapidly increasing costs and t...","`` 하지만, 급격히 증가하는 비용과 유로화의 강세는 업계 전체에 도전적이었고, 그..."
negative,"At this growth rate, paying off the national d...",이 정도 성장률이면 국가채무를 갚는 게 뼈아프다.
negative,Operating loss increased to EUR 17mn from a lo...,영업손실은 2005년 10.8 mn 손실에서 1700만 유로로 증가하였다.


In [ ]:
def analyze_article(prompt):
  try:
    prompt = f"""
              What is the sentiment of the following article text,
              which is delimited with triple backticks?

              Here is a list of two words that mean 'positive' and 'negative'.
              positive : ['up', 'raised', 'increased', 'was signed', 'risen', 'increasingly', 'increase', 'rose', 'profit went up','strengthens', 'growing', 'grow by', 'up from', '']
              negative : ['weak', 'loss', 'trouble', 'down', 'drop', 'dropped', 'inconsistent', 'low', 'negative', 'delayed', 'non-recurring', 'threatening', 'less', 'lower', 'overtime', 'fall', 'suffer', 'suffered', 'take the matter', 'fined', 'reduction', 'reduce', 'reductions', 'lay off']
              Identify the emotion in the text of the article by referring to the list of words.

              Give your answer as a  single word, either "positive" or "negative" or "netural".
              Just answer in lowercase.
              article text: ```{article}```

              """

    completion = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": prompt}],
            max_tokens=3,
            n=1,
            stop=None,
            temperature=0
        )

    response= completion.choices[0].message.content
    print(response)
    return response

  except client.error.RateLimitError as e:
    retry_time = e.retry_after if hasattr(e, 'retry_after') else 30
    print(f"Rate limit exceeded. Retrying in {retry_time} seconds...")
    time.sleep(retry_time)
    return analyze_article(article)

  except client.error.ServiceUnavailableError as e:
    retry_time = 10  # Adjust the retry time as needed
    print(f"Service is unavailable. Retrying in {retry_time} seconds...")
    time.sleep(retry_time)
    return analyze_article(article)

  except client.error.APIError as e:
    retry_time = e.retry_after if hasattr(e, 'retry_after') else 30
    print(f"API error occurred. Retrying in {retry_time} seconds...")
    time.sleep(retry_time)
    return analyze_article(article)

# 기사 분석해 저장하기
sentiments = []

for article in tqdm(df["sentence"]):
    sentiment = analyze_article(article)
    sentiments.append(sentiment)

df["Sentiment"] = sentiments

  0%|          | 1/501 [00:00<04:47,  1.74it/s]

neutral


  0%|          | 2/501 [00:01<04:26,  1.87it/s]

neutral


  1%|          | 3/501 [00:01<03:41,  2.24it/s]

...

negative


100%|█████████▉| 500/501 [03:19<00:00,  1.61it/s]

negative


100%|██████████| 501/501 [03:19<00:00,  2.51it/s]

negative


In [ ]:
df.to_csv('/content/drive/MyDrive/pknu/finance_data_test_output1.csv', index=False)

In [ ]:
# 데이터셋 불러오기
# 기존의 input과 result 자료를 합쳐서 새롭게 파일을 만들었음. 위의 파일과는 다른 파일임.
data = pd.read_csv('/content/drive/MyDrive/pknu/finance_data_test_output1.csv')


In [ ]:
# 첫 번째 행 삭제
data = data.iloc[:]
data=data.loc[:,['labels','sentence','Sentiment']]

# 결과 확인
data.head()

,labels,sentence,Sentiment
0,neutral,"According to Gran, the company has no plans to...",neutral
1,neutral,Technopolis plans to develop in stages an area...,positive
2,negative,The international electronic industry company ...,negative
3,positive,With the new production plant the company woul...,positive
4,positive,According to the company's updated strategy fo...,positive


In [ ]:
from sklearn.metrics import accuracy_score

# 정확도 계산
accuracy = accuracy_score(data['labels'], data['Sentiment'])
print("Accuracy:", accuracy)

Accuracy: 0.756
